In [5]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import sys
import glob
import os
import cv2
import time

from PIL import Image

from scipy.spatial import cKDTree
from scipy.ndimage import binary_fill_holes

In [6]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

using device: cuda


In [7]:
def find_black_area(image, window_size):
    h, w = image.shape
    max_density = -1
    best_coords = (0, 0)

    # Slide the window over the image
    for y in range(0, h - window_size[1] + 1, 1):
        for x in range(0, w - window_size[0] + 1, 1):
            # Extract the window from the image
            window = image[y:y + window_size[1], x:x + window_size[0]]

            # Count the number of black pixels
            black_pixel_count = np.sum(window <= 30)

            # Track the window with the maximum number of black pixels
            if black_pixel_count > max_density:
                max_density = black_pixel_count
                best_coords = (x, y)

    return best_coords, max_density

In [9]:
input_dir = "/mnt/c/Projects/Master/Data/Processed/2-LiveWingWingCropsImproved/"

# Find all jpg files
jpg_files = []
for root, _, files in os.walk(input_dir):
    for file in files:
        if file.endswith(".JPG") or file.endswith(".jpg"):
            jpg_files.append(os.path.join(root, file))

# Process every file
total_files = len(jpg_files)
digits = len(str(total_files))
for idx, jpg_file_path in enumerate(jpg_files, 1):
    jpg_basename = os.path.basename(jpg_file_path)
    relative_jpg_path = jpg_file_path.removeprefix(input_dir)
    print(f"Processing file {idx:0{digits}}/{total_files}:\t{relative_jpg_path}")
    
    # Process image
    image = cv2.imread(jpg_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray, (5, 5), 0)
    
    window_size = (20, 20)
    best_coords, max_density = find_black_area(blurred_image, window_size)

    print(max_density)
    
    # Plot the contour and the selected points
    plt.figure(figsize=(5, 5))
    plt.imshow(blurred_image, cmap="gray")
    plt.scatter(best_coords[0], best_coords[1], c="green", s=10)
    plt.axis("on")
    plt.show()


Processing file 0001/1147:	Round01-Hive01-2024_06_05-h01bee01.JPG


KeyboardInterrupt: 